In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import nltk
import random
from nltk.corpus import stopwords
from nltk.classify.scikitlearn import SklearnClassifier
import string
import pandas as pd

In [ ]:
df = pd.read_csv("./../input/spam.csv", encoding = 'latin-1')
print(df.head())  

Let's add messages and words in two different list for further use.

In [ ]:
message_list = []                                                        
word_list = []                                                           

for index, row in df.iterrows(): 
    
    #print(row['v1'], row['v2'])
    
    category = row['v1']
    message  = row['v2']
    
    message_list.append([message, category])
    
    for s in string.punctuation:                                        
        if s in message:
            message = message.replace(s, " ")
               
    stop = stopwords.words('english')
    for word in message.split(" "):                                        
        if not word in stop:
            word_list.append(word.lower())

In [ ]:
message_list[0:5]

Shuffle the message list to make sure we get the random items to predict.

In [ ]:
random.shuffle(message_list)

In [ ]:
word_list = nltk.FreqDist(word_list)

In [ ]:
print("words len : ", len(list(word_list.keys())))

In [ ]:
word_features = list(word_list.keys())

In [ ]:
def find_feature(word_features, message):
    feature = {}
    for word in word_features:
        feature[word] = word in message.lower()
    return feature

In [ ]:
featureset = [(find_feature(word_features, message), category) for (message, category) in message_list]

In [ ]:
trainingset = featureset[:int(len(featureset)*3/4)]
testingset = featureset[int(len(featureset)*3/4):]

In [ ]:
len(featureset)

In [ ]:
len(trainingset)

In [ ]:
len(testingset)

# # 1
**Naive Bayes Classifier:**

Let's try Naive Bayes Classifier and see what accuracy we can get.

In [ ]:
NBC = nltk.NaiveBayesClassifier.train(trainingset)

In [ ]:
nbc_accuracy = nltk.classify.accuracy(NBC, testingset)*100

In [ ]:
nbc_accuracy

We get 98.63% in the NB classifier.  It's a good sign! (Hope there is no issue in my data processing)

In [ ]:
NBC.show_most_informative_features(10)

Let's print some messages and their categories from NB Classifier:

In [ ]:
for message, category in message_list[0:10]:
    feature = find_feature(word_features, message)
    print(message, "-->", NBC.classify(feature))

# # 2
**Stochastic Gradient Descent Classifier:**

Let's try Stochastic Gradient Descent Classifier and see what accuracy we can get.

In [ ]:
from sklearn.linear_model import SGDClassifier

SGDC = SklearnClassifier(SGDClassifier())
SGDC.train(trainingset)

In [ ]:
feature = find_feature(word_features, message)
print(SGDC.classify(feature))

In [ ]:
print("SGD Classifier accuracy = " + str((nltk.classify.accuracy(SGDC, testingset))*100))

SGD accuracy is almost same as NB Classifier! Let's see the third model - Logistic Regression.

# # 3
**Logistic Regression classifier:**

Let's try Logistic Regression Classifier and see what accuracy we can get.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Logistic Regression classifier
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(trainingset)

In [ ]:
print("LRC accuracy: "+ str((nltk.classify.accuracy(LogisticRegression_classifier, testingset))*100))

Logistic Regression does the best so far. 

**Note:**

I haven't done any optimization in the code to keep it simple and plain. Feel free to fork and optimize the code if you want to.
If you find any issues in the data processing or classification, please let me know.